# Dense Retrieval for Entity Linking
Goal: evaluate pre-trained DR models for zero-shot EL on text

## 1. Load Corpus

In [10]:
# a subset of questions from https://github.com/askplatypus/wikidata-simplequestions
from beir.datasets.data_loader import GenericDataLoader

data_path = '/ivi/ilps/personal/svakule/spoken_qa/'


dataset = 'WD18/'

# query_path = data_path + dataset + "text_questions.jsonl"  # original text questions
query_path = data_path + dataset + "wav2vec2-base-960h.jsonl"  # questions transcribed from synthethised speech

corpus_path = data_path + dataset + "entities.jsonl"
qrels_path = data_path + dataset + "dev.tsv"


corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()

## 2. Evaluate with BEIR

In [11]:
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models

In [12]:
# DistilBERT v3 cosine winner!
model = DRES(models.SentenceBERT("msmarco-distilbert-base-v3"))
retriever = EvaluateRetrieval(model, score_function="cos_sim")
results1 = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results1, retriever.k_values)
precision['P@1']

0.43159

In [13]:
# TAS-B
model = DRES(models.SentenceBERT("msmarco-distilbert-base-tas-b"))
retriever = EvaluateRetrieval(model, score_function="dot")
results = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
precision['P@1']

0.43461

In [14]:
# DistilBERT dot
model = DRES(models.SentenceBERT("msmarco-distilbert-base-dot-prod-v3"))
retriever = EvaluateRetrieval(model, score_function="dot")
results1 = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results1, retriever.k_values)
precision['P@1']

0.41449

In [15]:
# DistilBERT v2
model = DRES(models.SentenceBERT("msmarco-distilbert-base-v2"))
retriever = EvaluateRetrieval(model, score_function="cos_sim")
results1 = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results1, retriever.k_values)
precision['P@1']

0.39537

In [16]:
# ANCE
model = DRES(models.SentenceBERT("msmarco-roberta-base-ance-fristp"))
retriever = EvaluateRetrieval(model, score_function="dot")
results = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
precision['P@1']

0.41549

## 3. Error Analysis

In [17]:
import random

top_k = 10
n_times = 1000

for _ in range(n_times):
    query_id, ranking_scores = random.choice(list(results1.items()))
    scores_sorted = sorted(ranking_scores.items(), key=lambda item: item[1], reverse=True)
    correct_id = list(qrels[query_id].keys())[0]
    correct_label = entities[correct_id]
    for rank in range(top_k):
        doc_id = scores_sorted[rank][0]
        if doc_id == correct_id and rank==0:
            break
        if corpus[doc_id].get("text") == correct_label and rank==0:
            break
        else:
            if rank==0:
                print("\nQuery : %s, %s" % (queries[query_id], correct_label))
            print("Rank %d: %s - %s" % (rank+1, doc_id, corpus[doc_id].get("text")))
            if doc_id == correct_id:
                print('Correct!')
#             break


Query : what shonere a film is iprahim lapiad, Ibrahim Labyad
Rank 1: Q7426511 - Sattam En Kaiyil
Rank 2: Q4661256 - Aahaa Enna Porutham
Rank 3: Q11900462 - Vappu Tuomioja
Rank 4: Q248892 - Iphigénie en Aulide
Rank 5: Q748941 - Pentti Hämäläinen
Rank 6: Q7278378 - Raadha Aur Seeta
Rank 7: Q1066446 - Sailor Moon S: The Movie
Rank 8: Q5686945 - Haym Soloveitchik
Rank 9: Q193338 - Danny Elfman
Rank 10: Q11865798 - Jalmari Parikka

Query : what is the classification of ernatini, Anatinae
Rank 1: Q2455704 - subfamily
Rank 2: Q2401491 - 21 Arietis
Rank 3: Q1361236 - Eroica
Rank 4: Q1073958 - Trochilinae
Rank 5: Q241087 - Alda Merini
Rank 6: Q384165 - 56 Arietis
Rank 7: Q43624 - Trochilidae
Rank 8: Q19338 - Pelecaniformes
Rank 9: Q1747898 - 37 Aquarii
Rank 10: Q11788 - Ursidae

Query : what is the language of the movy deacy showcase cat woman, DC Showcase: Catwoman
Rank 1: Q115760 - Catwoman
Rank 2: Q158952 - Catwoman
Rank 3: Q679630 - Slap Her... She's French
Rank 4: Q11855112 - DC Showcase

Rank 5: Q588594 - Cuban Amazon
Rank 6: Q11154007 - Ragaa Al Geddawy
Rank 7: Q234274 - Manosque
Rank 8: Q213132 - Manu Ginóbili
Rank 9: Q6309445 - Julio Mázzaro
Rank 10: Q580794 - Brütal Legend

Query : what label is the cordet signed with, The Chordettes
Rank 1: Q15453707 - Ken Corday
Rank 2: Q949100 - Alex Cord
Rank 3: Q4043197 - Universal Music Group Nashville
Rank 4: Q21077 - Warner Music Group
Rank 5: Q2338889 - MCA Records
Rank 6: Q38903 - Universal Music Group
Rank 7: Q7276309 - RED Ink Records
Rank 8: Q1239845 - Hellcat Records
Rank 9: Q770103 - Bertelsmann Music Group
Rank 10: Q7691534 - Team Love Records

Query : who did jewels vern influence, Jules Verne
Rank 1: Q229018 - Jewel
Rank 2: Q1688612 - Jewell Marceau
Rank 3: Q7921201 - Vereen Bell
Rank 4: Q1132943 - Queen: The eYe
Rank 5: Q20438283 - Mónika Veres
Rank 6: Q3106753 - Ginga Fukei Densetsu Sapphire
Rank 7: Q238215 - Natalie Merchant
Rank 8: Q12003 - Cher
Rank 9: Q2717834 - Wizardry II: The Knight of Diamonds
Rank 10: Q

Rank 5: Q6391749 - Kent Greenfield
Rank 6: Q2072023 - Irwin Dambrot
Rank 7: Q4755318 - Andrea West
Rank 8: Q6764439 - Marino Keith
Rank 9: Q73486 - Brooklyn Lee
Rank 10: Q5267942 - Devra Davis

Query : ho is the olther of bad hair day, Bad Hare Day
Rank 1: Q3496828 - Havin' a Bad Day
Rank 2: Q3221845 - Bad Hare Day
Correct!
Rank 3: Q1976535 - Bad Day L.A.
Rank 4: Q2252 - Buzz Aldrin
Rank 5: Q6740586 - Mal Brough
Rank 6: Q202741 - Blondie
Rank 7: Q5286116 - Do Me Bad Things
Rank 8: Q5353143 - Elahi Bux Soomro
Rank 9: Q7815970 - Tom Gorai
Rank 10: Q56759 - Olongapo

Query : is john pulsen a boy or a girl, John Polson
Rank 1: Q7259632 - Pulse
Rank 2: Q554833 - Pulse
Rank 3: Q638111 - Johntá Austin
Rank 4: Q2015601 - The Boy in Blue
Rank 5: Q543697 - John August
Rank 6: Q439841 - John Aniston
Rank 7: Q6257309 - John Sergeant
Rank 8: Q1332295 - John Metgod
Rank 9: Q794280 - John Small
Rank 10: Q3809316 - John Harris

Query : what position does not pledgeour play, Nap Lajoie
Rank 1: Q674953 

Rank 2: Q2706880 - Sal Maroni
Correct!
Rank 3: Q316431 - R. A. Salvatore
Rank 4: Q3945988 - Salvatore Capezio
Rank 5: Q53008 - Mauro Bolognini
Rank 6: Q6828233 - Michael Anthony Franano
Rank 7: Q1375738 - Stefano Bollani
Rank 8: Q5427800 - Fabio Borriello
Rank 9: Q574013 - Anthony Spilotro
Rank 10: Q4773138 - Anthony Morelli

Query : what was the reason of death for george cithen, Georges Couthon
Rank 1: Q1197442 - The Death of Mr. Lazarescu
Rank 2: Q5542900 - George Nicol
Rank 3: Q478352 - Workingman's Death
Rank 4: Q5544224 - George Saimes
Rank 5: Q5485838 - Frank Cimorelli
Rank 6: Q6249792 - James Yorkston
Rank 7: Q3376507 - Peter Cincotti
Rank 8: Q1380398 - Peter Cincotti
Rank 9: Q288337 - George Seaton
Rank 10: Q2314831 - The Deaths of Ian Stone

Query : is gaston gaparoche male or female, Gaston Gabaroche
Rank 1: Q2375926 - Eugène Vaulot
Rank 2: Q1349568 - Gaston Rahier
Rank 3: Q3099006 - Gaston Gabaroche
Correct!
Rank 4: Q535794 - Laurent Malet
Rank 5: Q298827 - Gaston Leroux
Ra